# API avec MLflow

In [6]:
# Import des objects créer par la modélisation
import joblib
pipeline=joblib.load('pipeline_scoring.joblib') # le modèle de prédiction sous forme de pipeline
signature=joblib.load('signature_MLflow.joblib') # la signature

In [7]:
# Générer le modèle MLflow

# Suppression de l'ancien modèle MLflow
import shutil

dossier_model_MLflow = r"./mlflow_model"

try:
    shutil.rmtree(dossier_model_MLflow)
except OSError as e:
    print("Dossier déjà supprimé !")
else:
    print("Ancien dossier du modèle supprimé !")

# Sauvegarde du pipeline à l'aide de la fonction save_model au format cloudpickle
import mlflow.sklearn
mlflow.sklearn.save_model(sk_model=pipeline, path='mlflow_model', serialization_format='cloudpickle',signature=signature)

Ancien dossier du modèle supprimé !


In [8]:
# Adaptation du modèle généré pour publication sur heroku
# heroku n'accepte pas conda /miniconda en librairie ou en add-on, cela provoque une erreur de build.
# Suppression de l'utilisation de conda dans le model MLflow

import os

# Suppression de 2 fichiers en trop
try :
    os.remove("./mlflow_model/conda.yaml")
    os.remove("./mlflow_model/requirements.txt") # le contenu de ce fichier a été ajoutée au requirements.txt situé à la racine de l'application
except OSError as e:
    print("Fichier(s) déjà supprimé(s)")
else :
    print("Fichiers requirementx.txt et cond.yaml supprimés !")

# Suppression de l'entrée conda dans le fichier MLmodel
with open("./mlflow_model/MLmodel", 'r') as fichier:
    contenu = fichier.readlines()
    with open("./mlflow_model/MLmodel_modif", 'w') as fichier_modif:
        for ligne in contenu :
            if ligne.find('conda') == -1 :
                fichier_modif.write(ligne)
            else :
                print("Entrée conda supprimée du fichier MLflow")
                
shutil.copy("./mlflow_model/MLModel_modif", "./mlflow_model/MLModel")
os.remove("./mlflow_model/MLmodel_modif")

Fichiers requirementx.txt et cond.yaml supprimés !
Entrée conda supprimée du fichier MLflow


## Serveur / API MLflow

In [9]:
# Dans  un terminal bash, lancer le serveur avec :  
# $ mlflow models serve -m ./Livrables/mlflow_model/

In [10]:
# Requête CURL pour interroger l'API

# Dans un terminal bash , interroger l'API  avec curl : 
# $ curl -v http://127.0.0.1:5000/invocations -H 'Content-Type: application/json' -d '{"columns": ["...","...",...], "data": [[...,...,...]]}'
# $ curl -v http://127.0.0.1:5000/invocations -H 'Content-Type: application/json' -d '{"columns": ["FLAG_OWN_CAR","FLAG_OWN_REALTY","CNT_CHILDREN","AMT_INCOME_TOTAL","AMT_CREDIT","AMT_ANNUITY","AMT_GOODS_PRICE","NAME_EDUCATION_TYPE","NAME_FAMILY_STATUS","DAYS_BIRTH","EXT_SOURCE_1","EXT_SOURCE_2","EXT_SOURCE_3"],"data": [["N","N","0","0","0","0","0","Secondary / secondary special","Single / not married","0",0.5,0.5,0.5]]}'